In [10]:
import os
import torch
import clip
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from random import shuffle
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import pandas as pd
if not os.path.exists("maws"):
    os.system("git clone https://github.com/facebookresearch/maws")
    os.chdir("maws")
    %pip install timm

In [11]:
from maws.maws.model_builder import build_model
from maws.maws.utils import start_inference_mode, predict_probs_and_plot

In [3]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

# Инициализация токенизатора и модели
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

# Пример текста
text = "The movie was fantastic and I enjoyed every moment of it."

# Преобразование текста в токены и подготовка для входа в модель
inputs = tokenizer(text, return_tensors="pt")
inputs

[2024-06-25 23:12:12,834] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/ubuntu/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[   0,  133, 1569,   21, 5500,    8,   38, 3776,  358, 1151,    9,   24,
            4,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [12]:
device = "cuda"
start_inference_mode(device=device)

In [13]:
def get_all_images(folder_path, extensions=(".jpg", ".jpeg", ".png")):
    image_paths = []
    for root, dirs, files in sorted(os.walk(folder_path)):
        dirs.sort()
        files.sort()
        for file in files:
            if file.lower().endswith(extensions):
                image_paths.append(os.path.join(root, file))
    return image_paths

# Функция для предобработки изображений
def preprocess_image(image_path):
    img = Image.open(image_path).convert("RGB")
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(img).unsqueeze(0)

In [14]:
filenames = []
target_labels = []
dir_path = '/home/ubuntu/Kate/clip_alex/clean_img/robbi'
directories = os.listdir(dir_path)

# Separate and exclude '.ipynb_checkpoints' if it exists

sorted_directories = sorted(directories, key=lambda x: int(x))

for idx, folder in enumerate(sorted_directories):
    folder_path = os.path.join(dir_path, folder)
    for filename in os.listdir(folder_path):
        if not filename.endswith((".jpg", ".jpeg", ".png")):
            continue
        filenames.append(os.path.join(folder_path, filename))
        target_labels.append(idx)

print("Target Labels:", target_labels[:6])
print("Filenames:", filenames[:6])

Target Labels: [0, 0, 0, 0, 0, 0]
Filenames: ['/home/ubuntu/Kate/clip_alex/clean_img/robbi/1/ROBBi x CHENPENG BUTTERMOON Co-brand Figures 1000%RED@鲨鱼小饱饱 with ROBBi#ROBBi #chenpeng #figures #ROBBiGLOBAL.jpg', '/home/ubuntu/Kate/clip_alex/clean_img/robbi/1/ROBBi x CHENPENG BUTTERMOON Co-brand Figures 1000%RED@鲨鱼小饱饱 with ROBBi#ROBBi #chenpeng #figures #ROBBiGLOBAL (3).jpg', '/home/ubuntu/Kate/clip_alex/clean_img/robbi/1/ROBBi x CHENPENG BUTTERMOON Mid-Autumn Festival theme co-brand series, the beauty of modern fashion, cross-bo.jpg', '/home/ubuntu/Kate/clip_alex/clean_img/robbi/1/ROBBi x CHENPENG BUTTERMOON Co-brand Figures 400%RED@玩具女侠 with ROBBi#ROBBi #chenpeng #figures #ROBBiGLOBAL (1).jpg', '/home/ubuntu/Kate/clip_alex/clean_img/robbi/1/ROBBi x CHENPENG BUTTERMOON Co-brand Figures 1000%RED@海豚Jessy有点辣 with ROBBi#ROBBi #chenpeng #figures #ROBBiGLOBAL (1).jpg', '/home/ubuntu/Kate/clip_alex/clean_img/robbi/1/ROBBi x CHENPENG BUTTERMOON Co-brand Figures 400%RED@玩具女侠 with ROBBi#ROBBi #chenp

In [15]:
descriptions = [
"The figure features a white and yellow color scheme, with yellow accents on its ears, arms, and boots. It has a transparent helmet with a hat on top, gloves draped like a scarf on the chest, and warm, padded boots on its feet.",
    "This figure has a red bird highlight on the helmet, sleek black glossy, red patterns on the sleeves and legs, Yin-Yang on the body",
    "The robot features a transparent head with a triangle and a ball in the center, creating focal point. Its color appears to change depending on the background and lighting, leading to varying light distortions. The body has a blue gradient towards the bottom.",
    "The robot has a glossy black finish with red accents and displays a 'B' logo on its chest.",
    "This robot is with a solid dark purple and burgundy color scheme, featuring sharp black protrusions extending from the sleeves. The body is adorned with a prominent emblem. The overall design is sleek with defined edges, emphasizing the sharp contrasts between the dark purple, burgundy, and black elements",
    "This figure is transparent with a gradient of pastel rainbow colors. Its helmet is filled with small colorful balls.",
    "The robot is white with a metallic finish on its helmet. It is depicted next to a sleek, futuristic bicycle.",
    "This robot is purple with a metallic sheen and is positioned beside a matching futuristic chair.",
    "The figure is transparent orange with intricate flower patterns embedded throughout its body. It has a glossy finish and the branding 'CREED 1760' is prominently displayed on its chest.",
    "This robot is made of transparent material with colored geometric patterns, including squares and rectangles, in primary colors such as red, blue, and yellow on the head and chest.",
    "The robot has a shiny golden finish with intricate floral patterns in various colors on its body and ears.",
    "This robot is predominantly green with white lines resembling tactical sports diagrams. It features a reflective helmet with 'LALIGA' written on it.",
    "This figure is dark green with black accents and yellow details. It has the 'Aston Martin Shanghai' logo on its chest.",
    "The robot is porcelain white body with blue Gzhel patterns on the sleeves, ear, shoes, and bottom of the helmet, emblem on the body.",
    "This pink robot is made of transparent, iridescent material with a pink heart emblem on its chest and helmet. Not yellow",
    "The robot is transparent body with a yelow hue, a pink heart inside the helmet, text on the body",
    "This white robot with a translucent tint, snow patterns, transparent helmet inside, various small shiny objects, white ear tips.",
    "The grey robot is metallic silver with black accents and features the Lamborghini logo on its chest.",
    "The robot is made of transparent purple material with a heart emblem on its chest and helmet,without triangle and pink",
    "The robot is completely gold with a highly reflective, shiny surface.",
    "The robot features a pink transparent head and ears, with a silver body. It has a pink heart emblem on its chest.",
    "This figure is red-orange, black helmet, red ears, matte.",
    "The robot has an iridescent, transparent body with a blue and red helmet and sole. blue NBA jersey with red stripes. It features NBA branding and a basketball jersey design.",
    "This figure is covered entirely with various NBA team stickers, creating a collage effect across its body and ears.",
    "The robot is transparent with yellow ears, arms, and shoes. It has red accents and a 'Peko' logo on its chest.",
    "The robot is black with a shiny silver helmet and a red visor line. It has a minimalist, futuristic design.",
    "This figure is entirely black with intricate patterns on its ears and chest. The body is matte and smooth, shiny patterns on the body and ears",
    "The robot features a gradient of colors, with red ears, a yellow body, turquoise legs, and purple shoes.",
    "The robot has a deep blue transparent body and helmet, giving it a sleek and glossy appearance.",
    "This figure is white with red and silver detailing. It has a clear, reflective helmet.",
    "The robot has a metallic blue reflective body and helmet. It features a clear visor with white patterns and the word 'Swift' on its chest. Not purple!",
    "The robot is transparent with pink ears, arms, and boots. It features various white line drawings and symbols on its body.Has no heart inside the helmet!not yellow",
    "The robot has an iridescent, rainbow-like color with predominant white and pink shades that change depending on the lighting, the transparent visor, a pink and green stripe on the helmet, and the 'Wilson' inscription on the chest.",
    "This figure is brown or bronze, made of wood.",
    "The robot is covered in long, white fur with a reflective visor."
]

labels = descriptions

# Загрузка модели MAWS-CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model = build_model("vit_h14_xlmr_l", "maws_clip").to(device)
#model = build_model("vit_2b14_xlmr_l", "maws_clip").to(device)

_ = model.eval()

CLIPConf(embed_dim=1024, vision_encoder_width=1280, text_encoder_width=1024)


In [16]:
def predict_top_k(model, image_path, texts, k=5):
    image = preprocess_image(image_path).to(device)
    
    with torch.no_grad():
        image_features = model.encode_images(image).float()
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features = model.encode_texts(texts).float()
        text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    top_k_indices = text_probs.cpu().topk(k, dim=-1).indices[0]
    
    return top_k_indices, text_probs[0, top_k_indices]

all_top_k_indices = []
all_true_labels = []

for image_path, true_label in zip(filenames, target_labels):
    top_k_indices, _ = predict_top_k(model, image_path, labels, k=5)
    all_top_k_indices.append(top_k_indices)
    all_true_labels.append(true_label)

top_1_predictions = [indices[0].item() for indices in all_top_k_indices]

#print(classification_report(all_true_labels, top_1_predictions, target_names=labels))

In [17]:
top_5_correct = []
for true_label, top_k in zip(all_true_labels, all_top_k_indices):
    top_5_correct.append(true_label in top_k)

total_images_per_category = Counter(all_true_labels)

# Calculate Top-5 accuracy per category
top_5_accuracy_per_category = {}
for label in total_images_per_category.keys():
    correct_predictions = sum(1 for i, correct in enumerate(top_5_correct) if all_true_labels[i] == label and correct)
    top_5_accuracy_per_category[label] = round((correct_predictions / total_images_per_category[label]) * 100, 2)

# Calculate overall Top-5 accuracy
overall_top_5_accuracy = round((sum(top_5_correct) / len(all_true_labels)) * 100, 2)

top_5_accuracy_table = []
for label, accuracy in top_5_accuracy_per_category.items():
    top_5_accuracy_table.append([label, labels[label], accuracy])

top_5_accuracy_df = pd.DataFrame(top_5_accuracy_table, columns=['Label Index', 'Label', 'Top-5 Accuracy'])
print(top_5_accuracy_df.sort_values(by="Top-5 Accuracy"))

print(f"\nOverall Top-5 Accuracy: {overall_top_5_accuracy}%")
large = top_5_accuracy_df.sort_values(by="Top-5 Accuracy")["Top-5 Accuracy"].values.astype(float)
top_5_accuracy_df.to_csv("top_5_accuracy_df1.csv", index=False)

    Label Index                                              Label  \
15           15  The robot is transparent body with a yelow hue...   
1             1  This figure has a red bird highlight on the he...   
27           27  The robot features a gradient of colors, with ...   
33           33      This figure is brown or bronze, made of wood.   
30           30  The robot has a metallic blue reflective body ...   
24           24  The robot is transparent with yellow ears, arm...   
18           18  The robot is made of transparent purple materi...   
14           14  This pink robot is made of transparent, irides...   
12           12  This figure is dark green with black accents a...   
32           32  The robot has an iridescent, rainbow-like colo...   
25           25  The robot is black with a shiny silver helmet ...   
21           21  This figure is red-orange, black helmet, red e...   
7             7  This robot is purple with a metallic sheen and...   
16           16  Thi